### Feathers in Focus: AML 2025 Kaggle Challenge 🪶
### Athina Papatriantafyllou • Alexandra Holíková • Buğra Sipahioğlu

### Setup Instructions
> **Setup Commands (in terminal):**
> ```bash
> python -m venv venv
> source venv/bin/activate
> pip install -r requirements.txt
> ```
> **Download the data and unzip it under "data/raw":** https://www.kaggle.com/competitions/aml-2025-feathers-in-focus/data

### Import the Libraries

In [1]:
# Import the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from PIL import ImageStat
from sklearn.model_selection import train_test_split
from torchvision.io import decode_image
from torch.utils.data import Dataset, WeightedRandomSampler
from torchvision.transforms import v2
import torch
from torch.utils.data import DataLoader
import itertools
import time
import torch
import torch.nn as nn

### Define the constants

In [ ]:
# Data path constants 
DATA_DIR = "data"
TRAIN_CSV_PATH = f"{DATA_DIR}/train_images.csv"
TEST_CSV_PATH = f"{DATA_DIR}/test_images_path.csv"
CLASS_NAMES_PATH = f"{DATA_DIR}/class_names.npy"
ATTRIBUTES_PATH = f"{DATA_DIR}/attributes.npy"
ATTRIBUTES_TXT_PATH = f"{DATA_DIR}/attributes.txt"
TRAIN_IMAGES_BASE_PATH = f"{DATA_DIR}/train_images"
TEST_IMAGES_BASE_PATH = f"{DATA_DIR}/test_images"

# Dataset constants
VAL_SPLIT_RATIO = 0.2  # 20% validation 80% test for instance. 
RANDOM_STATE = 45  # Keep the same random state across runs for reproducibility
NUM_CLASSES = 200 # It's given in the Kaggle competition description, so we don't need to count it (yet it's counted in EDA part)

# Image transformation constants
IMAGE_SIZE = 224  # Final image size after transformations
RESIZE_SIZE = 256  # Size before center crop for validation/test
RANDOM_CROP_SCALE = (0.7, 1.0)  # RandomResizedCrop scale range
HORIZONTAL_FLIP_PROB = 0.5  # RandomHorizontalFlip probability
ROTATION_DEGREES = 15  # RandomRotation degrees
COLOR_JITTER_BRIGHTNESS = 0.2
COLOR_JITTER_CONTRAST = 0.25
COLOR_JITTER_SATURATION = 0.25
COLOR_JITTER_HUE = 0.15

# Normalization constants (ImageNet normalization statistics since pretrained models are usually trained on ImageNet)
NORMALIZE_MEAN = [0.485, 0.456, 0.406]
NORMALIZE_STD = [0.229, 0.224, 0.225]

# Device constants
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu')
BATCH_SIZE_CPU = 8
BATCH_SIZE_GPU = 8
BATCH_SIZE_MPS = 8
NUM_WORKERS_CPU = 0
NUM_WORKERS_GPU = 4
NUM_WORKERS_MPS = 0
PIN_MEMORY_CPU = False
PIN_MEMORY_GPU = True
PIN_MEMORY_MPS = False

### Load the data

In [5]:
# Load the training and test sets
train_df = pd.read_csv(TRAIN_CSV_PATH)
test_df = pd.read_csv(TEST_CSV_PATH)
class_names = np.load(CLASS_NAMES_PATH, allow_pickle=True).item()

# Load attributes
attributes = np.load(ATTRIBUTES_PATH, allow_pickle=True)
with open(ATTRIBUTES_TXT_PATH, "r") as f:
    attribute_names = [line.strip().split(' ', 1)[1] for line in f.readlines()]

# Print the data summary
print("Loading the data....")
print(f"-Train Size: {len(train_df)} \n-Test Size: {len(test_df)} \n-Number of Classes: {len(class_names)}")
print(f"-Attributes: {len(attribute_names)} (shape: {attributes.shape if hasattr(attributes, 'shape') else 'N/A'})")


FileNotFoundError: [Errno 2] No such file or directory: '../data/train_images.csv'

### Explore the data

In [ ]:
# Inspect the training set by showing 10 random images
print("Random images (with their labels)from the training set:")
samples = train_df.sample(10)
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
for ax, (_, row) in zip(axes.flat, samples.iterrows()):
    img = Image.open(f"{TRAIN_IMAGES_BASE_PATH}{row['image_path']}")
    label_name = class_names.get(row['label'], f"Label: {row['label']}")
    ax.set_title(label_name, fontsize=10)
    ax.imshow(img)
plt.show()

In [ ]:
# Check image dimensions in the training set (concise version)
sizes = []
for p in train_df['image_path']:
    sizes.append(Image.open(f"{TRAIN_IMAGES_BASE_PATH}{p}").size)
widths = [s[0] for s in sizes]
heights = [s[1] for s in sizes]
print(f"Checking {len(widths)} images' dimensions: \n-Width: min={min(widths)}, max={max(widths)}, mean={np.mean(widths):.2f}\n-Height: min={min(heights)}, max={max(heights)}, mean={np.mean(heights):.2f}")

fig, axs = plt.subplots(1,2,figsize=(10,3))
axs[0].hist(widths, bins=20, color='skyblue'), axs[0].set(title='Image Widths', xlabel='Width', ylabel='Freq')
axs[1].hist(heights, bins=20, color='salmon'), axs[1].set(title='Image Heights', xlabel='Height', ylabel='Freq')
plt.tight_layout()
plt.show()

In [ ]:
# Inspect the attributes
attributes_df = pd.DataFrame({'Index': range(len(attribute_names)), 'Attribute Name': attribute_names})
print("Attributes:")
display(attributes_df.head(10))
print(f"Shape of attributes: {attributes.shape}")
print(f"Number of classes: {attributes.shape[0]}")
print(f"Number of attributes per class: {attributes.shape[1]}")
print(f"\nSample attribute names (first 10):")
for i, name in enumerate(attribute_names[:10]):
    print(f"  {i}: {name}")

# Show attributes for a sample class
sample_class_idx = 0
sample_class_name = class_names.get(sample_class_idx + 1, f"Class {sample_class_idx}")
sample_attrs = attributes[sample_class_idx]

print(f"\nSample: {sample_class_name}")
print(f"Attributes present (value=1):")
present_attrs = [attribute_names[i] for i in range(len(sample_attrs)) if sample_attrs[i] == 1]
print(f"Total: {len(present_attrs)} out of {len(sample_attrs)}")
print(f"First 10: {present_attrs[:10]}")

# Visualize attribute distribution
attr_counts = attributes.sum(axis=0)
plt.figure(figsize=(15, 5))
plt.hist(attr_counts, bins=30, edgecolor='black')
plt.xlabel('Number of classes with this attribute')
plt.ylabel('Frequency')
plt.title('Distribution of Attribute Occurrence Across Classes')
plt.show()

print(f"\nAttribute Statistics:")
print(f"Min occurrences: {attr_counts.min()}")
print(f"Max occurrences: {attr_counts.max()}")
print(f"Mean occurrences: {attr_counts.mean():.2f}")
print(f"Std occurrences: {attr_counts.std():.2f}")

In [ ]:
# Plot the class distribution to inspect the balance
class_counts = train_df['label'].value_counts().sort_index()
fig, ax = plt.subplots(figsize=(20, 10))

# Show the count for each class
ax.bar(range(len(class_counts)), class_counts.values, tick_label=None, edgecolor='k', alpha=0.7)
ax.set_xlabel('Class Index')
ax.set_ylabel('Number of Images')
ax.set_title('Number of Images per Class (All 200 Classes)')
ax.grid(axis='y')
plt.tight_layout()
plt.show()

# Statistics
print("Statistics:")
print(f"-Number of unique classes: {len(class_names)}")
print(f"-Average number of images per class: {class_counts.mean():.2f}")
print(f"-Standard deviation of images per class: {class_counts.std():.2f} (Lower the std, more balanced the dataset)")
print(f"-Minimum number of images per class: {class_counts.min()}")
print(f"-Maximum number of images per class: {class_counts.max()}")


In [ ]:
# Check for Corrupted Images

# Check Training Set
corrupt_counter_train = 0
for idx, row in train_df.iterrows():
    img_path = f"{TRAIN_IMAGES_BASE_PATH}{row['image_path']}"
    try:
        img = Image.open(img_path)
        img.verify()
    except Exception:
        corrupt_counter_train += 1

# Check Test Set
corrupt_counter_test = 0
for idx, row in test_df.iterrows():
    img_path = f"{TEST_IMAGES_BASE_PATH}{row['image_path']}"
    try:
        img = Image.open(img_path)
        img.verify()
    except Exception:
        corrupt_counter_test += 1

# Print the results
print(f"Total corrupt images in training set: {corrupt_counter_train}")
print(f"Total corrupt images in test set: {corrupt_counter_test}")


In [ ]:
# Check the color stats for all training images
def get_color_stats(image_path):
    img = Image.open(image_path).convert('RGB')
    stat = ImageStat.Stat(img)
    brightness = sum(stat.mean) / len(stat.mean)
    contrast = sum(stat.stddev) / len(stat.stddev)
    saturation = max(stat.mean) - min(stat.mean)
    return brightness, contrast, saturation

stats = [get_color_stats(f"{TRAIN_IMAGES_BASE_PATH}{p}") for p in train_df['image_path']]
brightnesses = [s[0] for s in stats]
contrasts = [s[1] for s in stats]
saturations = [s[2] for s in stats]

print(f"Color Statistics:")
print(f"- Brightness: mean={np.mean(brightnesses):.2f}, std={np.std(brightnesses):.2f}, range=[{min(brightnesses):.2f}, {max(brightnesses):.2f}]")
print(f"- Contrast: mean={np.mean(contrasts):.2f}, std={np.std(contrasts):.2f}, range=[{min(contrasts):.2f}, {max(contrasts):.2f}]")
print(f"- Saturation: mean={np.mean(saturations):.2f}, std={np.std(saturations):.2f}, range=[{min(saturations):.2f}, {max(saturations):.2f}]")
print("\nWISE WORDS FROM THE ML GODS:")
print("- High variance values indicate that our images already have diverse properties.\n- Hence, we don't need to use (high parameters for) ColorJitter.")

### Split the data into training, validation and test sets

In [ ]:
# Extract image paths and labels
X = train_df['image_path'].values
y = train_df['label'].values       

# Look at indices of labels (Y)
print("Before correction:")
print("- y indices min:", y.min())
print("- y indices max:", y.max())

# Since the labels start from 1, convert them to 0-indexed
y = y - 1

print("After correction:")
print("- y indices min:", y.min())
print("- y indices max:", y.max())

# Split the data into training and validation sets (80% train, 20% validation)
X_test = test_df['image_path'].values
X_train, X_val, y_train, y_val = train_test_split(
    X, y, 
    test_size = VAL_SPLIT_RATIO, 
    random_state = RANDOM_STATE, # keep the same random state across runs for reproducibility
    stratify = y  # Maintains class distribution in both sets
)

### Define the transformations

In [ ]:
# Transformations for training set
transformation_training = v2.Compose([

    # Randomly crop to IMAGE_SIZE from a resized area of the image.
    # - Scale controls how much of the original image area we keep (tunable via RANDOM_CROP_SCALE).
    # - Since feathers require fine details, we keep the scale high.
    # - Antialias=True is used to improve the quality of the resized image.
    v2.RandomResizedCrop(size=(IMAGE_SIZE, IMAGE_SIZE), scale=RANDOM_CROP_SCALE, antialias=True),
    
    # Random horizontal flip (tunable via HORIZONTAL_FLIP_PROB)
    v2.RandomHorizontalFlip(p=HORIZONTAL_FLIP_PROB), 
    
    # Random rotation (tunable via ROTATION_DEGREES)
    v2.RandomRotation(degrees=ROTATION_DEGREES), 
    
    # Color jitter (tunable via COLOR_JITTER constants)
    # Kept small since the stats of the training images are already diverse.
    # Strong color augmentations can distort the images and make the model more confused.
    v2.ColorJitter(
        brightness=COLOR_JITTER_BRIGHTNESS, 
        contrast=COLOR_JITTER_CONTRAST, 
        saturation=COLOR_JITTER_SATURATION, 
        hue=COLOR_JITTER_HUE
    ), 
    
    # Convert to float32 and scale to [0,1] (also converts to tensor)
    v2.ToDtype(torch.float32, scale=True),
    
    # Normalize the image (ImageNet stats for pretrained models, tunable)
    v2.Normalize(mean=NORMALIZE_MEAN, std=NORMALIZE_STD), 
])

# Transformations for validation set
transformation_validation = v2.Compose([
    
    # Resize to RESIZE_SIZE (no randomness for consistent validation)
    v2.Resize(RESIZE_SIZE, antialias=True),

    # Center crop to IMAGE_SIZE (matches the training set)
    v2.CenterCrop(IMAGE_SIZE),
    
    # Convert to float32 and scale to [0,1] (also converts to tensor)
    v2.ToDtype(torch.float32, scale=True),
     
    # Normalize the image (must match the training normalization)
    v2.Normalize(mean=NORMALIZE_MEAN, std=NORMALIZE_STD)
])

### Create Custom Dataset for PyTorch

In [ ]:
# Pytorch needs a custom dataset to load and transform the data
class FeatherImageDataset(Dataset):
    def __init__(self, image_paths, image_labels, transformation=None, target_transformation=None):
        self.image_paths = image_paths
        self.image_labels = image_labels
        self.transformation = transformation
        self.target_transformation = target_transformation
    
    def __len__(self):
        return len(self.image_labels)
    
    def __getitem__(self, i):
        image_path = self.image_paths[i]
        image_label = self.image_labels[i]  
        image = decode_image(image_path)
        if self.transformation:
            image = self.transformation(image)
        if self.target_transformation:
            image_label = self.target_transformation(image_label)
        return image, image_label

### Create Dataset Instances and DataLoaders

In [ ]:
# Create dataset instances with transformations
# Create custom dataset instance for training set
train_dataset = FeatherImageDataset(
    image_paths = [TRAIN_IMAGES_BASE_PATH + path for path in X_train],
    image_labels = y_train,
    transformation = transformation_training
)

# Create custom dataset instance for validation set
val_dataset = FeatherImageDataset(
    image_paths = [TRAIN_IMAGES_BASE_PATH + path for path in X_val],
    image_labels = y_val,
    transformation = transformation_validation
)

# Check device availability
# - If GPU is available, use it
# - If MPS is available, use it (GPU supportfor Apple Silicon)
# - Otherwise, use CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu')
if device.type == 'cuda':
    batch_size = BATCH_SIZE_GPU
    num_workers = NUM_WORKERS_GPU # Set to 0 for Jupyter notebooks as they might fail with too many workers
    pin_memory = PIN_MEMORY_GPU  # FYI: Pin memory speeds up the transfer data from CPU RAM to GPU vRAM
elif device.type == 'mps':
    batch_size = BATCH_SIZE_MPS
    num_workers = NUM_WORKERS_MPS
    pin_memory = PIN_MEMORY_MPS  # MPS doesn't support pin_memory
else:
    batch_size = BATCH_SIZE_CPU
    num_workers = NUM_WORKERS_CPU
    pin_memory = PIN_MEMORY_CPU  # Not needed for CPU (since it's used to speed up the transfer data from CPU RAM to GPU vRAM)

# Print the DataLoader configuration
print("DataLoader configuration:")
print(f"- Using device: {device}")
print(f"- Batch size: {batch_size}")
print(f"- Number of workers: {num_workers} (0 = main process, avoids pickling issues in Jupyter)")
print(f"- Pin memory: {pin_memory}")

In [ ]:
# Calculate sample weights for class-balanced sampling
# This addresses class imbalance by oversampling minority classes
print("Calculating class-balanced sample weights...")

# Count samples per class
class_counts = pd.Series(y_train).value_counts().sort_index()

# Calculate weights: inverse of class frequency
# Less frequent classes get higher weights
class_weights = 1.0 / class_counts
print(f"\nClass weight statistics:")
print(f"  Min weight: {class_weights.min():.4f} (most common class)")
print(f"  Max weight: {class_weights.max():.4f} (rarest class)")
print(f"  Mean weight: {class_weights.mean():.4f}")

# Assign a weight to each sample based on its class
sample_weights = [class_weights[label] for label in y_train]

# Create WeightedRandomSampler
# This will oversample minority classes during training
train_sampler = WeightedRandomSampler(
    weights=sample_weights,
    num_samples=len(sample_weights),
    replacement=True  # Allow oversampling (drawing samples multiple times)
)

print(f"\nClass-balanced sampler created:")
print(f"  Total samples per epoch: {len(sample_weights)}")
print(f"  Sampling with replacement: Yes")
print(f"  Effect: Minority classes will be seen more frequently during training")

In [ ]:
# Create DataLoader for training set with class-balanced sampling
# NOTE: sampler and shuffle are mutually exclusive - we use sampler instead of shuffle
train_loader = DataLoader(
    train_dataset,
    batch_size=16,
    sampler=train_sampler,  # Use weighted sampler for class balance (replaces shuffle=True)
    num_workers=num_workers,
    pin_memory=pin_memory
)

# Create DataLoader for validation set (no sampling - keep original distribution)
val_loader = DataLoader(
    val_dataset,
    batch_size=16,  # Use consistent batch size
    shuffle=False,  # Don't shuffle validation data: So that each time we use the validation, it's the same
    num_workers=num_workers,
    pin_memory=pin_memory  
)

# Print the final information
print(f"\nSetup:")
print(f"- Train dataset: {len(train_dataset)} samples")
print(f"- Validation dataset: {len(val_dataset)} samples")
print(f"- Batch size: 16")
print(f"- Training uses class-balanced sampling: Yes ✓")

### Birdy CNN

#### Load data, transform etc.

In [ ]:
# Import the model and training functions
from models.birdy_cnn import BirdyCNN
from train import train
from validate import validate

# Define the constants for augmentation
IMAGE_SIZE = 224
NUM_CLASSES = 200
IMAGE_SCALE = (0.8, 1.0)
IMAGE_ROTATION = 10
IMAGE_BRIGHTNESS = 0.2
IMAGE_CONTRAST = 0.2
IMAGE_FLIP = 0.5

# Set batch size to 16 (best parameter)
BATCH_SIZE_BEST = 16

# Define augmented transforms for training
transform_aug = v2.Compose([
    v2.RandomResizedCrop(size=(IMAGE_SIZE, IMAGE_SIZE), scale=IMAGE_SCALE),
    v2.RandomHorizontalFlip(p=IMAGE_FLIP),
    v2.RandomRotation(degrees=IMAGE_ROTATION),
    v2.ColorJitter(brightness=IMAGE_BRIGHTNESS, contrast=IMAGE_CONTRAST),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=NORMALIZE_MEAN, std=NORMALIZE_STD)
])

# Create training dataset with augmented transforms
train_dataset_aug = FeatherImageDataset(
    image_paths=[TRAIN_IMAGES_BASE_PATH + path for path in X_train],
    image_labels=y_train,
    transformation=transform_aug
)

# Create DataLoader for training with augmentation AND class-balanced sampling
# Reuse the same train_sampler we created earlier for class balance
train_loader_aug = DataLoader(
    train_dataset_aug,
    batch_size=BATCH_SIZE_BEST,  # Use best batch size of 16
    sampler=train_sampler,  # Use weighted sampler for class balance (replaces shuffle=True)
    num_workers=num_workers,
    pin_memory=pin_memory
)

print(f"Augmented training loader created with batch_size={BATCH_SIZE_BEST} and class-balanced sampling ✓")

#### Run the BIRDY model

In [ ]:
# Best hyperparameters
BEST_LR = 0.003
BEST_BATCH_SIZE = 16
BEST_EPOCHS = 60
BEST_DROPOUT = 0.3
BEST_WEIGHT_DECAY = 1e-5

# Initialize the BirdyCNN model with best dropout rate
model = BirdyCNN(image_size=IMAGE_SIZE, num_classes=NUM_CLASSES, dropout_rate=BEST_DROPOUT).to(DEVICE)

# Define the loss function and optimizer (SGD with weight decay for L2 regularization)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=BEST_LR, weight_decay=BEST_WEIGHT_DECAY)

print(f"Using device: {DEVICE}")
print(f"\nTraining BirdyCNN with BEST PARAMETERS:")
print(f"- Learning Rate: {BEST_LR}")
print(f"- Batch Size: {BEST_BATCH_SIZE}")
print(f"- Epochs: {BEST_EPOCHS}")
print(f"- Optimizer: SGD")
print(f"- Dropout Rate: {BEST_DROPOUT}")
print(f"- L2 Weight Decay: {BEST_WEIGHT_DECAY}")
print(f"\n- Training samples: {len(train_dataset_aug)}")
print(f"- Validation samples: {len(val_dataset)}")

# Train the model
trained_model = train(
    model=model,
    train_loader=train_loader_aug,
    val_loader=val_loader,
    criterion=criterion,
    optimizer=optimizer,
    device=DEVICE,
    num_epochs=BEST_EPOCHS
)

# Final validation
final_acc = validate(trained_model, val_loader, DEVICE)
print(f"\nFinal Validation Accuracy: {final_acc:.4f} ({final_acc*100:.2f}%)")